In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import numpy as np

In [2]:
pDir = './result/BU_ST_UD_QA/BU_ST_UD_QA/'

##### (1) 구성 dataset별 iou. dice score 비교

In [24]:
df = pd.read_csv(f'{pDir}/result.csv', index_col=0)
df.head()

,ID,task,iou,dice,SE,PE,F1,SP,ACC
0,BUSI_benign_147,BUSI,0.423321,0.594835,0.441215,0.912568,0.594835,0.999506,0.993057
1,BUSI_malignant_57,BUSI,0.737064,0.848632,0.752069,0.973645,0.848631,0.997933,0.975266
2,BUSI_benign_374,BUSI,0.750588,0.857527,0.996875,0.752358,0.857526,0.998390,0.998383
3,BUSI_malignant_196,BUSI,0.842667,0.914616,0.844318,0.997685,0.914616,0.999816,0.986496
4,BUSI_benign_241,BUSI,0.864250,0.927182,0.896400,0.960154,0.927182,0.998525,0.994629


In [29]:
df.groupby(by='task')['iou', 'dice'].agg(['mean', 'count'])

/tmp/ipykernel_340933/1947295277.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df.groupby(by='task')['iou', 'dice'].agg(['mean', 'count'])


iou            dice      
            mean count      mean count
task                                  
BUSI    0.728221   263  0.807609   263
QAMEBI  0.781357    94  0.864346    94
STU     0.785322    18  0.868262    18
UDIAT   0.784374    67  0.859961    67

In [ ]:
iDir = './inputs/TestSet/refine2/imagesTr'
lDir = './inputs/TestSet/refine2/labelsTr'
pDir = './predictions_T2_fold2'
os.makedirs('visualization',exist_ok=True)